In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [4]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [5]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [6]:
x=np.empty((12*471,9*18))
y=np.empty((12*471,1))

In [7]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=year_data[m][:,g:g+9].reshape(1,-1)
        y[m*471+g:m*471+g+1,:]=year_data[m][9,g+9]
        

- just test no normalize

In [8]:
import torch 
from torch import optim,nn

In [9]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [10]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [11]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [12]:
x1=torch.Tensor(x)
y1=torch.Tensor(y)

In [13]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 129.186523 , test loss : 136.567505
epoch = 2 train_loss : 91.456566 , test loss : 99.858284
epoch = 3 train_loss : 76.014778 , test loss : 83.505875
epoch = 4 train_loss : 71.316811 , test loss : 79.143204
epoch = 5 train_loss : 59.957409 , test loss : 66.713173
epoch = 6 train_loss : 57.076389 , test loss : 63.113998
epoch = 7 train_loss : 52.733757 , test loss : 58.500515
epoch = 9 train_loss : 50.595963 , test loss : 56.371109
epoch = 10 train_loss : 47.822048 , test loss : 52.793026
epoch = 11 train_loss : 44.997459 , test loss : 50.417725
epoch = 12 train_loss : 44.734993 , test loss : 49.856937
epoch = 14 train_loss : 43.259151 , test loss : 48.907909
epoch = 15 train_loss : 42.558681 , test loss : 47.375183
epoch = 16 train_loss : 40.457382 , test loss : 45.670235
epoch = 17 train_loss : 39.537884 , test loss : 45.058956
epoch = 18 train_loss : 39.331291 , test loss : 44.441578
epoch = 21 train_loss : 38.949688 , test loss : 44.177612
epoch = 24 train_los

epoch = 51 train_loss : 34.612225 , test loss : 35.977840
epoch = 54 train_loss : 34.914539 , test loss : 35.796871
epoch = 56 train_loss : 34.253906 , test loss : 35.600224
epoch = 64 train_loss : 34.041706 , test loss : 35.500839
epoch = 66 train_loss : 34.161835 , test loss : 35.278553
epoch = 71 train_loss : 33.670616 , test loss : 34.965538
epoch = 73 train_loss : 33.556187 , test loss : 34.941944
epoch = 77 train_loss : 33.399952 , test loss : 34.729050
epoch = 85 train_loss : 33.174728 , test loss : 34.534832
epoch = 89 train_loss : 33.504894 , test loss : 34.479115
epoch = 107 train_loss : 33.009270 , test loss : 34.271988
epoch = 119 train_loss : 32.964245 , test loss : 34.228230
epoch = 127 train_loss : 32.902618 , test loss : 34.161488
epoch = 145 train_loss : 32.835857 , test loss : 34.042061
epoch = 396 train_loss : 32.693550 , test loss : 34.040062
epoch = 431 train_loss : 32.638397 , test loss : 33.995911
epoch = 756 train_loss : 32.560535 , test loss : 33.973000
epoch =

In [17]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],512),nn.Linear(512,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 290.766388 , test loss : 269.512115
epoch = 2 train_loss : 103.084366 , test loss : 134.009445
epoch = 3 train_loss : 79.996941 , test loss : 101.958191
epoch = 4 train_loss : 70.435394 , test loss : 84.803787
epoch = 5 train_loss : 64.179726 , test loss : 74.874535
epoch = 6 train_loss : 59.422081 , test loss : 70.178734
epoch = 7 train_loss : 56.428925 , test loss : 67.311516
epoch = 8 train_loss : 54.308441 , test loss : 60.004467
epoch = 9 train_loss : 51.060757 , test loss : 57.502617
epoch = 10 train_loss : 50.022552 , test loss : 55.552086
epoch = 11 train_loss : 47.512028 , test loss : 53.037868
epoch = 12 train_loss : 46.209293 , test loss : 51.636555
epoch = 13 train_loss : 44.205700 , test loss : 51.578796
epoch = 15 train_loss : 45.089931 , test loss : 48.935425
epoch = 16 train_loss : 41.220982 , test loss : 47.141804
epoch = 20 train_loss : 38.471390 , test loss : 46.138603
epoch = 22 train_loss : 39.110271 , test loss : 44.030369
epoch = 24 train_l

epoch = 579 train_loss : 36.938469 , test loss : 21.137606
epoch = 842 train_loss : 36.743961 , test loss : 21.071686
epoch = 875 train_loss : 37.535267 , test loss : 20.861010
--------------------------------------------------------------------------
fold 4,train loss mean : 37.535267,test loss : 20.861010
-------------------------------------------------------------------------
epoch = 1 train_loss : 344.158264 , test loss : 364.149841
epoch = 2 train_loss : 98.619385 , test loss : 119.195351
epoch = 3 train_loss : 84.213676 , test loss : 94.371712
epoch = 4 train_loss : 73.286697 , test loss : 87.182747
epoch = 5 train_loss : 68.329010 , test loss : 76.775963
epoch = 6 train_loss : 62.501350 , test loss : 72.582199
epoch = 8 train_loss : 55.265423 , test loss : 63.680378
epoch = 9 train_loss : 56.973656 , test loss : 58.588451
epoch = 10 train_loss : 50.257824 , test loss : 55.361938
epoch = 12 train_loss : 46.380188 , test loss : 52.420948
epoch = 14 train_loss : 44.755901 , test l

### just test data col normalize

In [14]:
x.shape

(5652, 162)

In [15]:
x2=pd.DataFrame(x)
x2=x2.apply(lambda x:((x-x.mean())/x.std()))

In [16]:
x22=torch.Tensor(x2.values)
y22=torch.Tensor(y)

In [21]:
def get_net():
    return nn.Sequential(nn.Linear(x22.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x22,y22,256,0,0)

epoch = 1 train_loss : 563.576843 , test loss : 832.734558
epoch = 11 train_loss : 244.796875 , test loss : 767.408997
epoch = 12 train_loss : 214.190063 , test loss : 677.944275
epoch = 13 train_loss : 185.046494 , test loss : 552.179260
epoch = 14 train_loss : 155.866241 , test loss : 491.534088
epoch = 15 train_loss : 132.594147 , test loss : 383.075348
epoch = 16 train_loss : 107.768951 , test loss : 328.814972
epoch = 17 train_loss : 87.841248 , test loss : 224.141357
epoch = 18 train_loss : 71.900284 , test loss : 181.749496
epoch = 19 train_loss : 59.911762 , test loss : 129.574921
epoch = 20 train_loss : 51.394901 , test loss : 110.958374
epoch = 21 train_loss : 45.188522 , test loss : 86.687241
epoch = 22 train_loss : 41.066101 , test loss : 67.410683
epoch = 23 train_loss : 38.290638 , test loss : 59.066189
epoch = 24 train_loss : 37.328880 , test loss : 50.510220
epoch = 25 train_loss : 36.293625 , test loss : 48.489433
epoch = 26 train_loss : 35.523125 , test loss : 48.2970

epoch = 35 train_loss : 37.688641 , test loss : 23.398241
epoch = 39 train_loss : 37.174675 , test loss : 23.071039
epoch = 42 train_loss : 37.052330 , test loss : 22.730919
epoch = 47 train_loss : 36.647984 , test loss : 22.539539
epoch = 51 train_loss : 36.479469 , test loss : 22.492062
epoch = 72 train_loss : 35.717010 , test loss : 22.463718
epoch = 94 train_loss : 35.570087 , test loss : 22.332874
--------------------------------------------------------------------------
fold 4,train loss mean : 35.570087,test loss : 22.332874
-------------------------------------------------------------------------
epoch = 1 train_loss : 627.315674 , test loss : 616.007874
epoch = 2 train_loss : 572.924744 , test loss : 612.647827
epoch = 3 train_loss : 543.352234 , test loss : 608.106201
epoch = 4 train_loss : 514.450195 , test loss : 601.866272
epoch = 9 train_loss : 361.626282 , test loss : 592.471252
epoch = 10 train_loss : 326.927460 , test loss : 554.085449
epoch = 11 train_loss : 291.85397

In [23]:
def get_net():
    return nn.Sequential(nn.Linear(x22.shape[1],512),nn.Linear(512,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x22,y22,256,0,0)

epoch = 1 train_loss : 572.718445 , test loss : 584.010437
epoch = 2 train_loss : 522.648865 , test loss : 538.788757
epoch = 3 train_loss : 479.260925 , test loss : 500.751770
epoch = 4 train_loss : 431.364075 , test loss : 453.181488
epoch = 5 train_loss : 373.274139 , test loss : 394.611633
epoch = 6 train_loss : 308.810272 , test loss : 328.901825
epoch = 7 train_loss : 240.460709 , test loss : 254.985916
epoch = 8 train_loss : 176.963196 , test loss : 191.133621
epoch = 9 train_loss : 119.721054 , test loss : 129.870667
epoch = 10 train_loss : 78.511131 , test loss : 86.935173
epoch = 11 train_loss : 54.415157 , test loss : 61.995895
epoch = 12 train_loss : 43.039936 , test loss : 49.880802
epoch = 13 train_loss : 38.245274 , test loss : 44.676754
epoch = 14 train_loss : 37.054710 , test loss : 43.680313
epoch = 17 train_loss : 35.117310 , test loss : 42.466881
epoch = 18 train_loss : 34.847382 , test loss : 41.714108
epoch = 20 train_loss : 33.589920 , test loss : 40.671570
epoch

epoch = 31 train_loss : 35.138771 , test loss : 31.126564
epoch = 36 train_loss : 34.116096 , test loss : 31.032524
epoch = 40 train_loss : 34.017723 , test loss : 30.509760
epoch = 42 train_loss : 33.864189 , test loss : 30.366016
epoch = 44 train_loss : 33.771214 , test loss : 30.251110
epoch = 57 train_loss : 33.601959 , test loss : 30.230684
epoch = 90 train_loss : 33.839481 , test loss : 30.217751
epoch = 121 train_loss : 33.607899 , test loss : 29.801674
epoch = 177 train_loss : 33.617168 , test loss : 29.759855
--------------------------------------------------------------------------
fold 5,train loss mean : 33.617168,test loss : 29.759855
-------------------------------------------------------------------------
5 fold ,total train loss mean : 32.276692,total test loss mean : 34.778139 
-----------------------------------------------------------------------------


- just test linear model

In [17]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [18]:
n_folds=5

def rmlse_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x)
    rmse=np.sqrt(-cross_val_score(model,x,y,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [19]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
score=rmlse_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


Lasso score: 5.9364 (0.6516)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [20]:
np.sqrt(34.326048 )

5.8588435719005165

In [21]:
enet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9))
scoreo=rmlse_cv(enet)
print("\n enet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


 enet score: 5.9364 (0.6516)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [22]:
ridge=make_pipeline(RobustScaler(),Ridge(alpha=0.5))
score=rmlse_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


 ridge score: 5.9404 (0.6500)



In [23]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)
score = rmlse_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Gradient Boosting score: 6.9629 (1.0416)



In [24]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmlse_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[09:57:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:57:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:57:33] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [25]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmlse_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

LGBM score: 6.6617 (0.9785)



In [26]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data[i]))
all_train_data=pd.concat(month_list,axis=1).T

In [27]:
month_list[11].shape

(18, 480)

In [28]:
all_train_data.shape

(5760, 18)

- check reshape data is right ?

In [29]:
all_train_data.iloc[-5:,9:]

,9,10,11,12,13,14,15,16,17
475,37.0,0.0,76.0,2.6,1.9,109.0,97.0,1.0,1.4
476,28.0,0.0,80.0,2.2,1.9,108.0,107.0,1.7,1.3
477,17.0,0.0,82.0,2.3,1.9,114.0,118.0,1.5,1.6
478,24.0,0.0,84.0,2.3,2.0,108.0,100.0,2.0,1.8
479,29.0,0.0,84.0,2.3,2.0,109.0,105.0,2.0,2.0


In [30]:
all_train_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.527413,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.290152,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,-12.300000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


### correct outlier value

In [31]:
year_data1=year_data.copy()

In [32]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [33]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [34]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,6.700000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [35]:
for idx in [1,2,4,6,9,12,13]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >=0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
correct value between  : 1.8 1.8
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.12 

pm2.5 end ------------
correct value between  : 0.34 0.26
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 2.5 0.3
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 0.1 1.0
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -1.1 

pm2.5 end ------------
correct value between  : 6.6 1.2
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
---------

In [36]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [37]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534201,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.576035,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [38]:
x3=np.empty((12*471,18*9))
y3=np.empty((12*471,1))

In [39]:
for m in range(12):
    for g in range(471):
        x3[m*471+g:m*471+g+1,:]=year_data1[m][:,g:g+9].reshape(1,-1)
        y3[m*471+g:m*471+g+1,:]=year_data1[m][9,g+9]

In [40]:
year_data[11][-5,:5]

array([1.7, 1.7, 1.7, 1.7, 1.8])

In [41]:
year_data1[11][-5,:5]

array([1.7, 1.7, 1.7, 1.7, 1.8])

In [42]:
x[-5,-5:]

array([1.3, 1.7, 0.7, 0.4, 1.1])

In [43]:
x3[-5,-5:]

array([1.3, 1.7, 0.7, 0.4, 1.1])

In [44]:
y[-5:]

array([[37.],
       [28.],
       [17.],
       [24.],
       [29.]])

In [45]:
y3[-5:]

array([[37.],
       [28.],
       [17.],
       [24.],
       [29.]])

In [46]:
y[y<0]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [47]:
y3[y3<0]

array([], dtype=float64)

In [48]:
x3[x3<0]

array([], dtype=float64)

In [49]:
x33=pd.DataFrame(x3)
x33=x33.apply(lambda x: (x-x.mean())/x.std())
x33=x33.values
y33=y3

In [50]:
x333=torch.Tensor(x33)
y333=torch.Tensor(y33)

In [52]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 150.251907 , test loss : 155.604050
epoch = 2 train_loss : 99.558205 , test loss : 104.195030
epoch = 3 train_loss : 80.369507 , test loss : 87.102425
epoch = 4 train_loss : 69.669907 , test loss : 75.825897
epoch = 5 train_loss : 64.667076 , test loss : 70.746811
epoch = 6 train_loss : 61.921131 , test loss : 67.884300
epoch = 7 train_loss : 56.259201 , test loss : 62.566200
epoch = 8 train_loss : 52.487587 , test loss : 58.312618
epoch = 9 train_loss : 50.237362 , test loss : 55.953819
epoch = 10 train_loss : 48.078827 , test loss : 53.591438
epoch = 12 train_loss : 45.296902 , test loss : 50.552940
epoch = 13 train_loss : 43.835632 , test loss : 48.931911
epoch = 14 train_loss : 43.257683 , test loss : 48.668621
epoch = 15 train_loss : 43.066242 , test loss : 48.321022
epoch = 16 train_loss : 41.003517 , test loss : 46.407482
epoch = 18 train_loss : 40.249680 , test loss : 45.904747
epoch = 19 train_loss : 38.598541 , test loss : 44.173035
epoch = 20 train_los

epoch = 47 train_loss : 34.565628 , test loss : 35.519215
epoch = 52 train_loss : 34.032391 , test loss : 35.292900
epoch = 55 train_loss : 34.050674 , test loss : 35.128277
epoch = 58 train_loss : 33.804146 , test loss : 34.602654
epoch = 87 train_loss : 33.378014 , test loss : 34.423214
epoch = 92 train_loss : 33.236534 , test loss : 34.405964
epoch = 114 train_loss : 32.970219 , test loss : 34.267426
epoch = 125 train_loss : 32.875366 , test loss : 34.185539
epoch = 126 train_loss : 32.862984 , test loss : 34.093227
epoch = 155 train_loss : 33.150898 , test loss : 33.923546
epoch = 206 train_loss : 32.652462 , test loss : 33.831314
epoch = 599 train_loss : 32.596390 , test loss : 33.752548
--------------------------------------------------------------------------
fold 4,train loss mean : 32.596390,test loss : 33.752548
-------------------------------------------------------------------------
epoch = 1 train_loss : 217.226624 , test loss : 210.688721
epoch = 2 train_loss : 124.173622

In [63]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],512),nn.Linear(512,256),nn.Linear(256,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 242.862030 , test loss : 430.585968
epoch = 2 train_loss : 119.124214 , test loss : 118.581306
epoch = 3 train_loss : 92.216858 , test loss : 113.473755
epoch = 4 train_loss : 79.558067 , test loss : 93.636253
epoch = 5 train_loss : 68.957764 , test loss : 77.322014
epoch = 6 train_loss : 62.653736 , test loss : 67.041039
epoch = 7 train_loss : 55.060841 , test loss : 60.360199
epoch = 8 train_loss : 51.390102 , test loss : 59.180382
epoch = 9 train_loss : 50.936089 , test loss : 54.854618
epoch = 10 train_loss : 43.929916 , test loss : 50.223682
epoch = 12 train_loss : 41.247108 , test loss : 49.901432
epoch = 13 train_loss : 39.674068 , test loss : 46.727482
epoch = 16 train_loss : 40.105263 , test loss : 46.483082
epoch = 17 train_loss : 38.295731 , test loss : 44.586624
epoch = 19 train_loss : 40.744301 , test loss : 44.009396
epoch = 26 train_loss : 34.560661 , test loss : 41.944489
epoch = 28 train_loss : 34.315250 , test loss : 41.782665
epoch = 35 train_l

epoch = 49 train_loss : 34.928020 , test loss : 33.869534
epoch = 51 train_loss : 36.700760 , test loss : 33.787556
epoch = 63 train_loss : 35.367897 , test loss : 33.556740
epoch = 64 train_loss : 34.876911 , test loss : 32.767750
epoch = 76 train_loss : 34.153156 , test loss : 32.471138
epoch = 137 train_loss : 33.618958 , test loss : 32.308193
epoch = 166 train_loss : 34.511906 , test loss : 32.157074
epoch = 198 train_loss : 33.693707 , test loss : 31.952950
epoch = 460 train_loss : 33.335690 , test loss : 31.911142
epoch = 670 train_loss : 33.825115 , test loss : 31.892073
epoch = 784 train_loss : 33.554974 , test loss : 31.700590
--------------------------------------------------------------------------
fold 5,train loss mean : 33.554974,test loss : 31.700590
-------------------------------------------------------------------------
5 fold ,total train loss mean : 33.122111,total test loss mean : 34.370378 
--------------------------------------------------------------------------

In [105]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],8),nn.Linear(8,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 288.760529 , test loss : 299.105316
epoch = 2 train_loss : 234.510315 , test loss : 249.888062
epoch = 3 train_loss : 229.500900 , test loss : 245.100098
epoch = 4 train_loss : 221.382858 , test loss : 236.312546
epoch = 5 train_loss : 214.687164 , test loss : 229.509644
epoch = 6 train_loss : 208.009369 , test loss : 222.841476
epoch = 7 train_loss : 201.419632 , test loss : 216.145477
epoch = 8 train_loss : 195.092239 , test loss : 209.845413
epoch = 9 train_loss : 188.901154 , test loss : 203.525360
epoch = 10 train_loss : 183.050842 , test loss : 197.447571
epoch = 11 train_loss : 177.628250 , test loss : 191.980637
epoch = 12 train_loss : 172.545654 , test loss : 186.748230
epoch = 13 train_loss : 167.733047 , test loss : 181.838089
epoch = 14 train_loss : 163.276962 , test loss : 177.231750
epoch = 15 train_loss : 159.083923 , test loss : 172.846954
epoch = 16 train_loss : 155.173172 , test loss : 168.875824
epoch = 17 train_loss : 151.477036 , test loss : 

epoch = 150 train_loss : 53.291855 , test loss : 59.281574
epoch = 151 train_loss : 53.142475 , test loss : 59.100426
epoch = 152 train_loss : 52.649277 , test loss : 58.768467
epoch = 154 train_loss : 52.350552 , test loss : 58.456951
epoch = 155 train_loss : 52.252956 , test loss : 58.362350
epoch = 156 train_loss : 52.162903 , test loss : 58.284046
epoch = 157 train_loss : 51.896244 , test loss : 57.977318
epoch = 158 train_loss : 51.652313 , test loss : 57.675774
epoch = 159 train_loss : 51.408676 , test loss : 57.392944
epoch = 160 train_loss : 51.537674 , test loss : 57.357029
epoch = 161 train_loss : 51.573090 , test loss : 57.348331
epoch = 162 train_loss : 51.365749 , test loss : 57.123726
epoch = 163 train_loss : 50.794296 , test loss : 56.622768
epoch = 164 train_loss : 50.613033 , test loss : 56.541996
epoch = 165 train_loss : 50.449718 , test loss : 56.373264
epoch = 167 train_loss : 50.368111 , test loss : 56.317455
epoch = 168 train_loss : 50.050758 , test loss : 55.9693

epoch = 372 train_loss : 35.313892 , test loss : 41.124584
epoch = 374 train_loss : 35.264469 , test loss : 41.046535
epoch = 378 train_loss : 35.344063 , test loss : 41.030548
epoch = 380 train_loss : 35.121494 , test loss : 40.943882
epoch = 381 train_loss : 35.099327 , test loss : 40.893944
epoch = 384 train_loss : 35.084538 , test loss : 40.880508
epoch = 385 train_loss : 35.015724 , test loss : 40.830517
epoch = 390 train_loss : 34.911461 , test loss : 40.754025
epoch = 393 train_loss : 34.921089 , test loss : 40.717049
epoch = 394 train_loss : 34.929749 , test loss : 40.702599
epoch = 398 train_loss : 34.828613 , test loss : 40.648743
epoch = 399 train_loss : 34.788113 , test loss : 40.614304
epoch = 402 train_loss : 34.666615 , test loss : 40.573494
epoch = 403 train_loss : 34.759628 , test loss : 40.570953
epoch = 407 train_loss : 34.578041 , test loss : 40.525467
epoch = 410 train_loss : 34.514271 , test loss : 40.450233
epoch = 412 train_loss : 34.477901 , test loss : 40.4234

epoch = 33 train_loss : 132.719315 , test loss : 150.769745
epoch = 34 train_loss : 130.729111 , test loss : 148.753815
epoch = 35 train_loss : 128.893021 , test loss : 146.882385
epoch = 36 train_loss : 127.122955 , test loss : 145.417603
epoch = 37 train_loss : 125.385361 , test loss : 143.578232
epoch = 38 train_loss : 123.784668 , test loss : 141.973450
epoch = 39 train_loss : 122.241714 , test loss : 140.430420
epoch = 40 train_loss : 120.738068 , test loss : 138.911957
epoch = 41 train_loss : 119.343369 , test loss : 137.266495
epoch = 42 train_loss : 117.946274 , test loss : 135.876160
epoch = 43 train_loss : 116.632790 , test loss : 134.478088
epoch = 44 train_loss : 115.374565 , test loss : 133.228760
epoch = 45 train_loss : 114.149895 , test loss : 132.010483
epoch = 46 train_loss : 112.949539 , test loss : 130.518661
epoch = 47 train_loss : 111.775475 , test loss : 129.395218
epoch = 48 train_loss : 110.656464 , test loss : 128.275238
epoch = 49 train_loss : 109.603485 , tes

epoch = 190 train_loss : 52.563210 , test loss : 63.858692
epoch = 191 train_loss : 52.012016 , test loss : 63.544781
epoch = 192 train_loss : 51.908978 , test loss : 63.377678
epoch = 193 train_loss : 51.687553 , test loss : 63.199482
epoch = 194 train_loss : 51.647705 , test loss : 63.124939
epoch = 195 train_loss : 51.241505 , test loss : 62.935001
epoch = 196 train_loss : 51.204723 , test loss : 62.718201
epoch = 200 train_loss : 50.391014 , test loss : 62.119194
epoch = 202 train_loss : 50.053406 , test loss : 61.835754
epoch = 203 train_loss : 49.898346 , test loss : 61.615543
epoch = 204 train_loss : 49.752811 , test loss : 61.579170
epoch = 205 train_loss : 49.604626 , test loss : 61.341122
epoch = 206 train_loss : 49.455776 , test loss : 61.180130
epoch = 207 train_loss : 49.252289 , test loss : 61.040607
epoch = 209 train_loss : 48.973236 , test loss : 60.742294
epoch = 212 train_loss : 48.471134 , test loss : 60.406094
epoch = 213 train_loss : 48.356258 , test loss : 60.1751

...

epoch = 487 train_loss : 35.597206 , test loss : 36.901684
epoch = 490 train_loss : 35.576408 , test loss : 36.825768
epoch = 497 train_loss : 35.461536 , test loss : 36.748646
epoch = 502 train_loss : 35.384346 , test loss : 36.659729
epoch = 505 train_loss : 35.301750 , test loss : 36.654308
epoch = 510 train_loss : 35.232712 , test loss : 36.568283
epoch = 516 train_loss : 35.179352 , test loss : 36.493225
epoch = 518 train_loss : 35.110737 , test loss : 36.465855
epoch = 520 train_loss : 35.109180 , test loss : 36.414772
epoch = 527 train_loss : 34.985950 , test loss : 36.382778
epoch = 530 train_loss : 35.096497 , test loss : 36.330425
epoch = 535 train_loss : 34.882980 , test loss : 36.274750
epoch = 539 train_loss : 34.835377 , test loss : 36.255936
epoch = 540 train_loss : 34.821186 , test loss : 36.238926
epoch = 541 train_loss : 34.932709 , test loss : 36.196182
epoch = 543 train_loss : 34.821442 , test loss : 36.136360
epoch = 548 train_loss : 34.744690 , test loss : 36.1308

epoch = 5 train_loss : 396.441620 , test loss : 404.982910
epoch = 6 train_loss : 376.913208 , test loss : 383.944366
epoch = 7 train_loss : 358.417664 , test loss : 363.824524
epoch = 8 train_loss : 340.220520 , test loss : 344.203705
epoch = 9 train_loss : 323.078064 , test loss : 325.789948
epoch = 10 train_loss : 306.856445 , test loss : 308.005585
epoch = 11 train_loss : 291.486450 , test loss : 291.461334
epoch = 12 train_loss : 276.987213 , test loss : 275.986847
epoch = 13 train_loss : 263.574585 , test loss : 261.539612
epoch = 14 train_loss : 251.216446 , test loss : 248.182434
epoch = 15 train_loss : 239.573395 , test loss : 235.837234
epoch = 16 train_loss : 228.946091 , test loss : 224.446442
epoch = 17 train_loss : 218.886887 , test loss : 213.751282
epoch = 18 train_loss : 209.740936 , test loss : 204.051254
epoch = 19 train_loss : 201.189774 , test loss : 194.963959
epoch = 20 train_loss : 193.402969 , test loss : 186.712082
epoch = 21 train_loss : 186.186508 , test los

epoch = 154 train_loss : 57.896088 , test loss : 52.823601
epoch = 155 train_loss : 57.765366 , test loss : 52.701195
epoch = 156 train_loss : 57.627930 , test loss : 52.567177
epoch = 157 train_loss : 57.258923 , test loss : 52.236595
epoch = 158 train_loss : 57.063034 , test loss : 52.029739
epoch = 160 train_loss : 56.713505 , test loss : 51.714340
epoch = 161 train_loss : 56.550419 , test loss : 51.585358
epoch = 162 train_loss : 56.344017 , test loss : 51.347023
epoch = 163 train_loss : 56.203003 , test loss : 51.241497
epoch = 164 train_loss : 55.990208 , test loss : 51.014557
epoch = 165 train_loss : 55.882462 , test loss : 50.916065
epoch = 166 train_loss : 55.645386 , test loss : 50.672001
epoch = 167 train_loss : 55.465515 , test loss : 50.508686
epoch = 169 train_loss : 55.153076 , test loss : 50.194958
epoch = 170 train_loss : 54.980690 , test loss : 50.051109
epoch = 171 train_loss : 54.810341 , test loss : 49.901672
epoch = 172 train_loss : 54.632904 , test loss : 49.6869

epoch = 360 train_loss : 38.758865 , test loss : 34.370003
epoch = 362 train_loss : 38.718620 , test loss : 34.329189
epoch = 367 train_loss : 38.554642 , test loss : 34.290344
epoch = 368 train_loss : 38.502808 , test loss : 34.209679
epoch = 369 train_loss : 38.466888 , test loss : 34.142532
epoch = 370 train_loss : 38.544647 , test loss : 34.124893
epoch = 374 train_loss : 38.326870 , test loss : 34.001205
epoch = 377 train_loss : 38.343723 , test loss : 33.930786
epoch = 378 train_loss : 38.280632 , test loss : 33.901962
epoch = 381 train_loss : 38.141670 , test loss : 33.873417
epoch = 382 train_loss : 38.161694 , test loss : 33.793835
epoch = 383 train_loss : 38.176174 , test loss : 33.756092
epoch = 387 train_loss : 38.038662 , test loss : 33.658596
epoch = 389 train_loss : 37.925968 , test loss : 33.607292
epoch = 392 train_loss : 37.840664 , test loss : 33.583981
epoch = 393 train_loss : 37.848736 , test loss : 33.490124
epoch = 395 train_loss : 37.777935 , test loss : 33.4673

epoch = 1053 train_loss : 33.906631 , test loss : 29.837877
epoch = 1105 train_loss : 33.858658 , test loss : 29.825220
epoch = 1109 train_loss : 33.825672 , test loss : 29.814447
epoch = 1114 train_loss : 33.879551 , test loss : 29.804144
epoch = 1129 train_loss : 33.852291 , test loss : 29.797880
epoch = 1138 train_loss : 33.803692 , test loss : 29.797783
epoch = 1148 train_loss : 33.808681 , test loss : 29.787867
epoch = 1160 train_loss : 33.797993 , test loss : 29.775429
epoch = 1166 train_loss : 33.788143 , test loss : 29.753630
epoch = 1207 train_loss : 33.749950 , test loss : 29.733604
epoch = 1224 train_loss : 33.772785 , test loss : 29.706675
epoch = 1256 train_loss : 33.734791 , test loss : 29.686760
epoch = 1315 train_loss : 33.775188 , test loss : 29.678291
epoch = 1354 train_loss : 33.750854 , test loss : 29.672173
epoch = 1376 train_loss : 33.668762 , test loss : 29.658360
epoch = 1390 train_loss : 33.677979 , test loss : 29.647678
epoch = 1418 train_loss : 33.658054 , te

In [51]:
n_folds=5

def rmlse_cv2(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x33)
    rmse=np.sqrt(-cross_val_score(model,x33,y33,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [52]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.005,random_state=1))
score=rmlse_cv2(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 5.6020 (0.5464)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


- use boxcox transformer to gaussian dist

In [53]:
from scipy.stats import boxcox,boxcox_normmax
from scipy.special import boxcox1p

In [54]:
all_train_data1.shape

(5760, 18)

In [55]:
all_train_data1.shape[1]

18

In [56]:
for i in range(all_train_data1.shape[1]):
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
1 --skew :  -5.095255676523535 --kurt :  85.70760213466164
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
4 --skew :  5.378836580918865 --kurt :  41.25921290333162
5 --skew :  1.6391191931961593 --kurt :  3.8457481859587697
6 --skew :  2.071649558193799 --kurt :  6.419180514549189
7 --skew :  1.3121217461933794 --kurt :  5.2975502420733696
8 --skew :  1.103678593463204 --kurt :  1.7728459308764268
9 --skew :  1.2763236528649795 --kurt :  2.115391100082106
10 --skew :  21.7839204262489 --kurt :  623.8917732792679
11 --skew :  -0.6050425772235472 --kurt :  -0.1979575284081454
12 --skew :  2.1806637991938813 --kurt :  8.332926208370576
13 --skew :  -0.7973228533500623 --kurt :  21.5544747714386
14 --skew :  0.5569055078560654 --kurt :  -0.9777533530993172
15 --skew :  0.5166061431100419 --kurt :  -1.0455161114586577
16 --skew :  0.9251663396039612 --kurt :  1

In [57]:
all_train_data2=pd.DataFrame()
for i in range(all_train_data1.shape[1]):
    lm=boxcox_normmax(all_train_data1[i]+1)
    all_train_data2[i]=boxcox1p(all_train_data1[i],lm)
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())
    print('lambda : ',lm)
    print(all_train_data2.index[i],'--skew : ',all_train_data2[i].skew(),'--kurt : ',all_train_data2[i].kurt())
    print('--------------------------------------------------------------------')

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
lambda :  1.461645538288768
0 --skew :  -0.07414275326700835 --kurt :  -0.9258821506244836
--------------------------------------------------------------------
1 --skew :  -5.095255676523535 --kurt :  85.70760213466164
lambda :  5.990940784090655
1 --skew :  0.2052435896115669 --kurt :  0.583625236412292
--------------------------------------------------------------------
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
lambda :  -3.259330902857795
2 --skew :  -0.003907562716963148 --kurt :  -0.06972890461562153
--------------------------------------------------------------------
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
lambda :  -6.182620739009405
3 --skew :  0.05859262861967747 --kurt :  -0.5568735662256521
--------------------------------------------------------------------
4 --skew :  5.378836580918865 --kurt :  41.25921290333162
lambda :  -0.7826645126628972
4 --skew :  -0.03484059806052529 --

In [58]:
all_train_data2.shape

(5760, 18)

In [59]:
year_data2={}
for m in range(12):
    month_data2=np.empty((18,480))
    month_data2[:,:]=all_train_data2.T.iloc[:,m*480:(m+1)*480]
    year_data2[m]=month_data2
    

In [60]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data1.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

In [61]:
year_data[11][-6,-5:]

array([2.6, 2.2, 2.3, 2.3, 2.3])

In [62]:
year_data3[11][-6,-5:]

array([2.6, 2.2, 2.3, 2.3, 2.3])

In [63]:
year_data2[11][-6,-5:]

array([1.1003593 , 1.01289662, 1.03598957, 1.03598957, 1.03598957])

### compare all_train_data1 and all_train_data2
- all_train_data1 : just correct outlier value
- all_train_data2 : just boxcox after all_train_data1

In [64]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534201,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.576035,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [65]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,65.832202,0.186377,0.079806,0.672361,2.283694,1.937404,7.449352,9.327724,5.756380,0.008977,15482.226219,1.051691,3.676586,12.803597,11.687312,1.046624,0.813470
std,25.536335,12.405338,0.041562,0.032614,0.176061,0.523410,0.306804,2.431622,3.291614,2.725299,0.035324,6063.422584,0.303361,0.481140,3.602422,3.020925,0.263948,0.292340
min,12.833239,0.000000,0.068067,0.000000,0.000000,0.000000,0.769032,0.000000,0.000000,0.000000,0.000000,1616.845054,0.000000,0.000000,0.096879,0.000000,0.000000,0.000000
25%,49.926550,50.952479,0.158556,0.055291,0.562804,1.938097,1.735287,5.766358,7.204513,3.853588,0.000000,10617.557930,0.852464,3.245618,9.751980,9.184518,0.857234,0.592611
50%,70.525274,63.921527,0.188618,0.076901,0.672848,2.270782,1.916995,7.472206,9.283713,5.763730,0.000000,15534.166404,1.035990,3.534372,12.099014,11.107693,1.042283,0.818152
75%,88.521007,79.522794,0.215417,0.103613,0.775804,2.651342,2.143833,9.063998,11.523345,7.603404,0.000000,20398.022735,1.243713,3.834142,15.383128,13.963600,1.233421,1.030980
max,133.380091,120.311749,0.306532,0.160805,1.192886,3.876332,2.905076,20.678088,20.721569,14.709463,0.161645,30295.187454,2.192628,7.863631,18.965279,16.819218,1.852950,1.618322


## diff with hw_test81
- all_train_data3 : normalized
- all_train_data2 : no normalized

In [66]:
all_train_data3=all_train_data2.copy()
all_train_data3=all_train_data3.apply(lambda x : (x-x.mean())/x.std())

In [67]:
all_train_data3.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,1.019875e-13,-5.144712e-14,6.653925e-14,2.549689e-14,2.334629e-15,-6.727368e-14,-4.221841e-14,2.062856e-15,5.215490e-14,1.772519e-14,-1.659610e-15,6.245795e-14,-9.682569e-15,-1.537578e-14,1.616595e-14,-8.086471e-15,3.354384e-14,3.181464e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.255345e+00,-5.306764e+00,-2.846587e+00,-2.447008e+00,-3.818917e+00,-4.363104e+00,-3.808196e+00,-3.063532e+00,-2.833784e+00,-2.112201e+00,-2.541260e-01,-2.286725e+00,-3.466795e+00,-7.641412e+00,-3.527271e+00,-3.868786e+00,-3.965265e+00,-2.782619e+00
25%,-8.027755e-01,-1.199461e+00,-6.693840e-01,-7.516859e-01,-6.222681e-01,-6.602794e-01,-6.587790e-01,-6.921282e-01,-6.450364e-01,-6.981957e-01,-2.541260e-01,-8.022974e-01,-6.567335e-01,-8.957229e-01,-8.471015e-01,-8.284860e-01,-7.175293e-01,-7.554874e-01
50%,3.868283e-03,-1.540203e-01,5.390355e-02,-8.905258e-02,2.767230e-03,-2.466757e-02,-6.652185e-02,9.398760e-03,-1.337080e-02,2.696880e-03,-2.541260e-01,8.566150e-03,-5.175865e-02,-2.955772e-01,-1.955858e-01,-1.918681e-01,-1.644871e-02,1.601343e-02
75%,7.085792e-01,1.103605e+00,6.987095e-01,7.299705e-01,5.875442e-01,7.024097e-01,6.728357e-01,6.640201e-01,6.670347e-01,6.777326e-01,-2.541260e-01,8.107297e-01,6.329803e-01,3.274645e-01,7.160547e-01,7.535070e-01,7.077023e-01,7.440291e-01
max,2.465256e+00,4.391621e+00,2.890982e+00,2.483615e+00,2.956513e+00,3.042810e+00,3.154037e+00,5.440292e+00,3.461476e+00,3.285175e+00,4.321989e+00,2.443003e+00,3.760984e+00,8.702351e+00,1.710428e+00,1.698787e+00,3.054866e+00,2.753135e+00


In [68]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,65.832202,0.186377,0.079806,0.672361,2.283694,1.937404,7.449352,9.327724,5.756380,0.008977,15482.226219,1.051691,3.676586,12.803597,11.687312,1.046624,0.813470
std,25.536335,12.405338,0.041562,0.032614,0.176061,0.523410,0.306804,2.431622,3.291614,2.725299,0.035324,6063.422584,0.303361,0.481140,3.602422,3.020925,0.263948,0.292340
min,12.833239,0.000000,0.068067,0.000000,0.000000,0.000000,0.769032,0.000000,0.000000,0.000000,0.000000,1616.845054,0.000000,0.000000,0.096879,0.000000,0.000000,0.000000
25%,49.926550,50.952479,0.158556,0.055291,0.562804,1.938097,1.735287,5.766358,7.204513,3.853588,0.000000,10617.557930,0.852464,3.245618,9.751980,9.184518,0.857234,0.592611
50%,70.525274,63.921527,0.188618,0.076901,0.672848,2.270782,1.916995,7.472206,9.283713,5.763730,0.000000,15534.166404,1.035990,3.534372,12.099014,11.107693,1.042283,0.818152
75%,88.521007,79.522794,0.215417,0.103613,0.775804,2.651342,2.143833,9.063998,11.523345,7.603404,0.000000,20398.022735,1.243713,3.834142,15.383128,13.963600,1.233421,1.030980
max,133.380091,120.311749,0.306532,0.160805,1.192886,3.876332,2.905076,20.678088,20.721569,14.709463,0.161645,30295.187454,2.192628,7.863631,18.965279,16.819218,1.852950,1.618322


In [69]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data3.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

## diff with hw_test81

In [86]:
x4=np.empty((12*471,18*9))
y4=np.empty((12*471,1))

for m in range(12):
    for g in range(471):
        x4[m*471+g:m*471+g+1,:]=year_data3[m][:,g:g+9].reshape(1,-1)
        y4[m*471+g:m*471+g+1,:]=year_data3[m][9,g+9]

In [87]:
x4[-6,81:92]

array([-0.79645675, -0.79645675, -0.60553382, -1.27240954, -1.8175533 ,
       -1.8175533 , -0.90134018,  0.3641883 ,  1.00139281, -0.25412602,
       -0.25412602])

In [88]:
x4[-5,81:92]

array([-0.79645675, -0.60553382, -1.27240954, -1.8175533 , -1.8175533 ,
       -0.90134018,  0.3641883 ,  1.00139281,  1.48322449, -0.25412602,
       -0.25412602])

In [89]:
x4[-4,81:92]

array([-0.60553382, -1.27240954, -1.8175533 , -1.8175533 , -0.90134018,
        0.3641883 ,  1.00139281,  1.48322449,  1.00139281, -0.25412602,
       -0.25412602])

In [90]:
y4.shape

(5652, 1)

In [91]:
y4[-6:]

array([[ 1.48322449],
       [ 1.00139281],
       [ 0.57751679],
       [-0.06379855],
       [ 0.3641883 ],
       [ 0.6281137 ]])

In [92]:
y4[-5:]

array([[ 1.00139281],
       [ 0.57751679],
       [-0.06379855],
       [ 0.3641883 ],
       [ 0.6281137 ]])

In [93]:
y4[-4:]

array([[ 0.57751679],
       [-0.06379855],
       [ 0.3641883 ],
       [ 0.6281137 ]])

In [94]:
x44=torch.Tensor(x4)
y44=torch.Tensor(y4)

In [90]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

epoch = 1 train_loss : 0.354327 , test loss : 0.390271
epoch = 2 train_loss : 0.288429 , test loss : 0.314791
epoch = 3 train_loss : 0.249911 , test loss : 0.273869
epoch = 4 train_loss : 0.221793 , test loss : 0.242339
epoch = 5 train_loss : 0.205295 , test loss : 0.224097
epoch = 6 train_loss : 0.189097 , test loss : 0.208025
epoch = 7 train_loss : 0.177969 , test loss : 0.197278
epoch = 8 train_loss : 0.172280 , test loss : 0.187166
epoch = 9 train_loss : 0.172585 , test loss : 0.186293
epoch = 10 train_loss : 0.161396 , test loss : 0.176225
epoch = 11 train_loss : 0.154276 , test loss : 0.169861
epoch = 12 train_loss : 0.149747 , test loss : 0.163218
epoch = 14 train_loss : 0.150314 , test loss : 0.162989
epoch = 15 train_loss : 0.146769 , test loss : 0.159080
epoch = 17 train_loss : 0.148672 , test loss : 0.158506
epoch = 18 train_loss : 0.147083 , test loss : 0.158389
epoch = 21 train_loss : 0.143416 , test loss : 0.156425
epoch = 23 train_loss : 0.140625 , test loss : 0.153385
e

In [91]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],256),nn.Linear(256,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

epoch = 1 train_loss : 0.317243 , test loss : 0.342729
epoch = 2 train_loss : 0.239142 , test loss : 0.263027
epoch = 3 train_loss : 0.211435 , test loss : 0.228256
epoch = 4 train_loss : 0.190771 , test loss : 0.207209
epoch = 5 train_loss : 0.178426 , test loss : 0.195883
epoch = 6 train_loss : 0.168668 , test loss : 0.184204
epoch = 7 train_loss : 0.166250 , test loss : 0.182232
epoch = 8 train_loss : 0.158647 , test loss : 0.169413
epoch = 9 train_loss : 0.152368 , test loss : 0.165382
epoch = 10 train_loss : 0.149241 , test loss : 0.161780
epoch = 12 train_loss : 0.148300 , test loss : 0.160614
epoch = 14 train_loss : 0.148268 , test loss : 0.158498
epoch = 17 train_loss : 0.142127 , test loss : 0.153723
epoch = 20 train_loss : 0.141504 , test loss : 0.152053
epoch = 33 train_loss : 0.140899 , test loss : 0.151999
epoch = 40 train_loss : 0.140036 , test loss : 0.151443
epoch = 43 train_loss : 0.138927 , test loss : 0.151213
epoch = 44 train_loss : 0.139499 , test loss : 0.149247
e

In [92]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],16),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,1000,0.0005,5,x44,y44,256,0,0)

epoch = 1 train_loss : 0.500259 , test loss : 0.519400
epoch = 2 train_loss : 0.412532 , test loss : 0.430099
epoch = 3 train_loss : 0.379156 , test loss : 0.402438
epoch = 4 train_loss : 0.355301 , test loss : 0.378293
epoch = 5 train_loss : 0.337266 , test loss : 0.361386
epoch = 6 train_loss : 0.322658 , test loss : 0.346927
epoch = 7 train_loss : 0.310615 , test loss : 0.337034
epoch = 8 train_loss : 0.298743 , test loss : 0.325149
epoch = 9 train_loss : 0.288892 , test loss : 0.315243
epoch = 10 train_loss : 0.278711 , test loss : 0.305200
epoch = 11 train_loss : 0.271411 , test loss : 0.299324
epoch = 12 train_loss : 0.262257 , test loss : 0.288823
epoch = 13 train_loss : 0.255143 , test loss : 0.282591
epoch = 14 train_loss : 0.248568 , test loss : 0.275522
epoch = 15 train_loss : 0.241594 , test loss : 0.268819
epoch = 16 train_loss : 0.235531 , test loss : 0.261964
epoch = 17 train_loss : 0.233309 , test loss : 0.258607
epoch = 18 train_loss : 0.224293 , test loss : 0.250026
e

epoch = 15 train_loss : 0.225175 , test loss : 0.213945
epoch = 16 train_loss : 0.220176 , test loss : 0.208216
epoch = 17 train_loss : 0.214336 , test loss : 0.206950
epoch = 18 train_loss : 0.209063 , test loss : 0.198202
epoch = 19 train_loss : 0.204218 , test loss : 0.196655
epoch = 20 train_loss : 0.200515 , test loss : 0.195240
epoch = 21 train_loss : 0.195628 , test loss : 0.190096
epoch = 22 train_loss : 0.191101 , test loss : 0.185611
epoch = 23 train_loss : 0.187525 , test loss : 0.181712
epoch = 24 train_loss : 0.183711 , test loss : 0.179018
epoch = 25 train_loss : 0.180987 , test loss : 0.174807
epoch = 26 train_loss : 0.177771 , test loss : 0.173400
epoch = 27 train_loss : 0.175270 , test loss : 0.172191
epoch = 28 train_loss : 0.172424 , test loss : 0.168780
epoch = 29 train_loss : 0.169567 , test loss : 0.167939
epoch = 30 train_loss : 0.167697 , test loss : 0.165412
epoch = 31 train_loss : 0.165859 , test loss : 0.163985
epoch = 32 train_loss : 0.163104 , test loss : 0

epoch = 64 train_loss : 0.142514 , test loss : 0.143034
epoch = 74 train_loss : 0.142705 , test loss : 0.142481
epoch = 75 train_loss : 0.140711 , test loss : 0.142381
epoch = 88 train_loss : 0.140849 , test loss : 0.141006
epoch = 91 train_loss : 0.140534 , test loss : 0.139904
--------------------------------------------------------------------------
fold 5,train loss mean : 0.140534,test loss : 0.139904
-------------------------------------------------------------------------
5 fold ,total train loss mean : 0.138157,total test loss mean : 0.146397 
-----------------------------------------------------------------------------


In [93]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],32),nn.Linear(32,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x44,y44,256,0,0)

epoch = 1 train_loss : 0.381629 , test loss : 0.399963
epoch = 2 train_loss : 0.293572 , test loss : 0.312060
epoch = 3 train_loss : 0.261381 , test loss : 0.282909
epoch = 4 train_loss : 0.237863 , test loss : 0.260051
epoch = 5 train_loss : 0.221804 , test loss : 0.240736
epoch = 6 train_loss : 0.207388 , test loss : 0.226529
epoch = 7 train_loss : 0.201104 , test loss : 0.221888
epoch = 8 train_loss : 0.187545 , test loss : 0.205953
epoch = 9 train_loss : 0.179314 , test loss : 0.197147
epoch = 10 train_loss : 0.175913 , test loss : 0.189993
epoch = 12 train_loss : 0.167336 , test loss : 0.185310
epoch = 13 train_loss : 0.159530 , test loss : 0.175348
epoch = 14 train_loss : 0.156213 , test loss : 0.170752
epoch = 15 train_loss : 0.153288 , test loss : 0.166733
epoch = 16 train_loss : 0.151911 , test loss : 0.164389
epoch = 18 train_loss : 0.147388 , test loss : 0.159558
epoch = 19 train_loss : 0.146350 , test loss : 0.159420
epoch = 22 train_loss : 0.149217 , test loss : 0.159242
e

epoch = 14 train_loss : 0.175547 , test loss : 0.167802
epoch = 16 train_loss : 0.166967 , test loss : 0.162763
epoch = 17 train_loss : 0.162563 , test loss : 0.158650
epoch = 18 train_loss : 0.157606 , test loss : 0.154868
epoch = 19 train_loss : 0.159768 , test loss : 0.151749
epoch = 22 train_loss : 0.152614 , test loss : 0.148502
epoch = 23 train_loss : 0.149784 , test loss : 0.147605
epoch = 26 train_loss : 0.149103 , test loss : 0.147063
epoch = 27 train_loss : 0.150687 , test loss : 0.147041
epoch = 29 train_loss : 0.145986 , test loss : 0.144530
epoch = 44 train_loss : 0.142213 , test loss : 0.141785
epoch = 65 train_loss : 0.140175 , test loss : 0.141721
epoch = 70 train_loss : 0.140223 , test loss : 0.141248
epoch = 73 train_loss : 0.140910 , test loss : 0.141089
epoch = 83 train_loss : 0.140804 , test loss : 0.139512
epoch = 378 train_loss : 0.140996 , test loss : 0.139362
--------------------------------------------------------------------------
fold 5,train loss mean : 0.1

#### set lr = 0.0001 , batch_size = 64 , test loss <=1

In [1]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],16),nn.ReLU(),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x44,y44,64,0,0)

NameError: name 'nn' is not defined

In [95]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],8),nn.ReLU(),nn.Linear(8,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x44,y44,64,0,0)

epoch = 1 train_loss : 0.703851 , test loss : 0.723326
epoch = 2 train_loss : 0.561331 , test loss : 0.585175
epoch = 3 train_loss : 0.492634 , test loss : 0.519692
epoch = 4 train_loss : 0.454616 , test loss : 0.483981
epoch = 5 train_loss : 0.429250 , test loss : 0.460158
epoch = 6 train_loss : 0.409460 , test loss : 0.441180
epoch = 7 train_loss : 0.393354 , test loss : 0.425830
epoch = 8 train_loss : 0.379418 , test loss : 0.412674
epoch = 9 train_loss : 0.367053 , test loss : 0.401016
epoch = 10 train_loss : 0.355819 , test loss : 0.390081
epoch = 11 train_loss : 0.345574 , test loss : 0.380364
epoch = 12 train_loss : 0.336413 , test loss : 0.371739
epoch = 13 train_loss : 0.327649 , test loss : 0.362754
epoch = 14 train_loss : 0.319542 , test loss : 0.354904
epoch = 15 train_loss : 0.311942 , test loss : 0.347495
epoch = 16 train_loss : 0.304844 , test loss : 0.340931
epoch = 17 train_loss : 0.298329 , test loss : 0.334615
epoch = 18 train_loss : 0.292040 , test loss : 0.328828
e

epoch = 194 train_loss : 0.117731 , test loss : 0.162437
epoch = 197 train_loss : 0.117568 , test loss : 0.161968
epoch = 211 train_loss : 0.116103 , test loss : 0.161848
epoch = 216 train_loss : 0.115401 , test loss : 0.161605
epoch = 218 train_loss : 0.115328 , test loss : 0.161537
epoch = 220 train_loss : 0.115236 , test loss : 0.161385
epoch = 224 train_loss : 0.114628 , test loss : 0.161285
epoch = 227 train_loss : 0.114350 , test loss : 0.161242
epoch = 232 train_loss : 0.113871 , test loss : 0.161222
epoch = 242 train_loss : 0.113008 , test loss : 0.161131
epoch = 245 train_loss : 0.112759 , test loss : 0.161078
epoch = 272 train_loss : 0.111313 , test loss : 0.160960
--------------------------------------------------------------------------
fold 1,train loss mean : 0.111313,test loss : 0.160960
-------------------------------------------------------------------------
epoch = 1 train_loss : 0.630170 , test loss : 0.641322
epoch = 2 train_loss : 0.523336 , test loss : 0.542347
ep

epoch = 172 train_loss : 0.117997 , test loss : 0.172827
epoch = 174 train_loss : 0.117630 , test loss : 0.172819
epoch = 181 train_loss : 0.116951 , test loss : 0.172752
epoch = 182 train_loss : 0.116715 , test loss : 0.172351
epoch = 184 train_loss : 0.117295 , test loss : 0.172228
epoch = 185 train_loss : 0.116256 , test loss : 0.172043
epoch = 186 train_loss : 0.116300 , test loss : 0.171440
epoch = 194 train_loss : 0.115404 , test loss : 0.170744
epoch = 211 train_loss : 0.113645 , test loss : 0.170540
epoch = 213 train_loss : 0.113665 , test loss : 0.170418
epoch = 219 train_loss : 0.113181 , test loss : 0.169849
epoch = 235 train_loss : 0.111848 , test loss : 0.169648
epoch = 270 train_loss : 0.109707 , test loss : 0.169623
--------------------------------------------------------------------------
fold 2,train loss mean : 0.109707,test loss : 0.169623
-------------------------------------------------------------------------
epoch = 1 train_loss : 0.846754 , test loss : 0.858632


epoch = 135 train_loss : 0.128916 , test loss : 0.165151
epoch = 136 train_loss : 0.128497 , test loss : 0.164958
epoch = 137 train_loss : 0.128157 , test loss : 0.164678
epoch = 139 train_loss : 0.127695 , test loss : 0.163780
epoch = 140 train_loss : 0.127509 , test loss : 0.163505
epoch = 141 train_loss : 0.127251 , test loss : 0.163357
epoch = 145 train_loss : 0.126415 , test loss : 0.163124
epoch = 146 train_loss : 0.126099 , test loss : 0.163027
epoch = 148 train_loss : 0.125832 , test loss : 0.162584
epoch = 149 train_loss : 0.125787 , test loss : 0.162564
epoch = 150 train_loss : 0.125745 , test loss : 0.162313
epoch = 154 train_loss : 0.124483 , test loss : 0.161981
epoch = 155 train_loss : 0.124475 , test loss : 0.161489
epoch = 158 train_loss : 0.124135 , test loss : 0.161389
epoch = 168 train_loss : 0.122041 , test loss : 0.161278
epoch = 170 train_loss : 0.121886 , test loss : 0.161270
epoch = 171 train_loss : 0.121670 , test loss : 0.160916
epoch = 172 train_loss : 0.1215

### linear function  test

In [95]:
def rmsle_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x4)
    rmse=np.sqrt(-cross_val_score(model,x4,y4,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [96]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
score=rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.3851 (0.0225)



In [97]:
enet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9))
scoreo=rmsle_cv(enet)
print("\n enet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


 enet score: 0.3851 (0.0225)



In [98]:
ridge=make_pipeline(RobustScaler(),Ridge(alpha=0.5))
score=rmsle_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


 ridge score: 0.3876 (0.0225)



In [99]:
ridge=make_pipeline(Ridge(alpha=0.5))
score=rmsle_cv(ridge)
print("\n ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


 ridge score: 0.3876 (0.0224)



In [100]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Gradient Boosting score: 0.4412 (0.0296)



In [101]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[10:11:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:11:48] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:11:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [106]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

LGBM score: 0.4184 (0.0266)



In [ ]:
# x4_cxv=pd.DataFrame(x4.reshape(-1,9))
# x4_cxv.to_csv(r'F:\study\ml\HonyiLee2020\hw1\data\wx_x_cxv.csv')

In [107]:
x4[-5,81:91]

array([-0.79645675, -0.60553382, -1.27240954, -1.8175533 , -1.8175533 ,
       -0.90134018,  0.3641883 ,  1.00139281,  1.48322449, -0.25412602])

In [108]:
x4[-4,81:91]

array([-0.60553382, -1.27240954, -1.8175533 , -1.8175533 , -0.90134018,
        0.3641883 ,  1.00139281,  1.48322449,  1.00139281, -0.25412602])

In [109]:
x4[-3,81:91]

array([-1.27240954, -1.8175533 , -1.8175533 , -0.90134018,  0.3641883 ,
        1.00139281,  1.48322449,  1.00139281,  0.57751679, -0.25412602])

In [110]:
y4[-5:]

array([[ 1.00139281],
       [ 0.57751679],
       [-0.06379855],
       [ 0.3641883 ],
       [ 0.6281137 ]])

In [111]:
all_train_data3.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,1.019875e-13,-5.144712e-14,6.653925e-14,2.549689e-14,2.334629e-15,-6.727368e-14,-4.221841e-14,2.062856e-15,5.215490e-14,1.772519e-14,-1.659610e-15,6.245795e-14,-9.682569e-15,-1.537578e-14,1.616595e-14,-8.086471e-15,3.354384e-14,3.181464e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.255345e+00,-5.306764e+00,-2.846587e+00,-2.447008e+00,-3.818917e+00,-4.363104e+00,-3.808196e+00,-3.063532e+00,-2.833784e+00,-2.112201e+00,-2.541260e-01,-2.286725e+00,-3.466795e+00,-7.641412e+00,-3.527271e+00,-3.868786e+00,-3.965265e+00,-2.782619e+00
25%,-8.027755e-01,-1.199461e+00,-6.693840e-01,-7.516859e-01,-6.222681e-01,-6.602794e-01,-6.587790e-01,-6.921282e-01,-6.450364e-01,-6.981957e-01,-2.541260e-01,-8.022974e-01,-6.567335e-01,-8.957229e-01,-8.471015e-01,-8.284860e-01,-7.175293e-01,-7.554874e-01
50%,3.868283e-03,-1.540203e-01,5.390355e-02,-8.905258e-02,2.767230e-03,-2.466757e-02,-6.652185e-02,9.398760e-03,-1.337080e-02,2.696880e-03,-2.541260e-01,8.566150e-03,-5.175865e-02,-2.955772e-01,-1.955858e-01,-1.918681e-01,-1.644871e-02,1.601343e-02
75%,7.085792e-01,1.103605e+00,6.987095e-01,7.299705e-01,5.875442e-01,7.024097e-01,6.728357e-01,6.640201e-01,6.670347e-01,6.777326e-01,-2.541260e-01,8.107297e-01,6.329803e-01,3.274645e-01,7.160547e-01,7.535070e-01,7.077023e-01,7.440291e-01
max,2.465256e+00,4.391621e+00,2.890982e+00,2.483615e+00,2.956513e+00,3.042810e+00,3.154037e+00,5.440292e+00,3.461476e+00,3.285175e+00,4.321989e+00,2.443003e+00,3.760984e+00,8.702351e+00,1.710428e+00,1.698787e+00,3.054866e+00,2.753135e+00


In [112]:
 0.6281137*2.725299+5.756380

7.4681776384963

In [113]:
from scipy.special import inv_boxcox1p

In [114]:
inv_boxcox1p(7.4681776384963,0.4145678510523434)

28.999999855599405

In [115]:
-0.06379855*2.725299+5.756380

5.5825098754835505

In [116]:
inv_boxcox1p(5.5825098754835505,0.4145678510523434)

17.000000800694327

In [117]:
0.38*0.38

0.1444

In [118]:
x4_len=x4.shape[0]
print(int(0.8*x4_len))
x4_train=x4[:int(0.8*x4_len),:]
y4_train=y4[:int(0.8*x4_len),:]
x4_test=x4[int(0.8*x4_len):,:]
y4_test=y4[int(0.8*x4_len):,:]

4521


In [119]:
x4_train.shape

(4521, 162)

In [120]:
y4_train.shape

(4521, 1)

In [121]:
lasso=Lasso(alpha=0.0005,random_state=1)
lasso.fit(x4_train,y4_train)
y4_hat=lasso.predict(x4_test)

In [122]:
y4_hat[-5:]

array([1.14616487, 0.81584775, 0.2574312 , 0.04508735, 0.34347021])

In [123]:
y4_test[-5:]

array([[ 1.00139281],
       [ 0.57751679],
       [-0.06379855],
       [ 0.3641883 ],
       [ 0.6281137 ]])

In [124]:
0.04508735*2.725299+5.756380

inv_boxcox1p(5.87925650986765,0.4145678510523434)

5.87925650986765

In [126]:
from sklearn.model_selection import cross_val_predict

In [146]:
def rmsle_cv_predict(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x4_train)
    rmse=np.sqrt(-cross_val_predict(model,x4_train,y4_train,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [134]:
lasso=Lasso(alpha=0.0005,random_state=1)
lasso.fit(x4_train,y4_train)
y4_hat=lasso.predict(x4_test).reshape(-1,1)
np.sqrt(np.sum(np.power(y4_hat-y4_test,2),axis=0)/y4_test.shape[0])

array([0.39542851])

In [135]:
y4_hat_orig=y4_hat*2.725299+5.756380
y4_test_orig=y4_test*2.725299+5.756380
y4_hat_orig=inv_boxcox1p(y4_hat_orig,0.4145678510523434)
y4_test_orig=inv_boxcox1p(y4_test_orig,0.4145678510523434)
np.sqrt(np.sum(np.power(y4_hat_orig-y4_test_orig,2),axis=0)/y4_test.shape[0])

array([5.51428678])

In [139]:
inv_boxcox1p(1.05,0.4145678510523434)

1.3909286462303396

In [157]:
ridge=Ridge(alpha=0.5)
ridge.fit(x4_train,y4_train)
y_hat=ridge.predict(x4_test)

In [158]:
y_hat[-5:]

array([[1.16289152],
       [0.78155887],
       [0.25523402],
       [0.07791427],
       [0.35014741]])

In [168]:
x44_train=torch.Tensor(x4_train)
y44_train=torch.Tensor(y4_train)
x44_test=torch.Tensor(x4_test)

In [169]:
def train_kfold_pred(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd,test_features):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')
    preds=net(test_features).detach().numpy()
    print('pred :\n', preds[-5:])

    return preds

In [170]:
def get_net():
    return nn.Sequential(nn.Linear(x44.shape[1],8),nn.ReLU(),nn.Linear(8,1))
net1=get_net()
y4_hat=train_kfold_pred(net1,5000,0.0001,5,x44_train,y44_train,64,0,0,x44_test)

epoch = 1 train_loss : 0.974191 , test loss : 0.991456
epoch = 2 train_loss : 0.741834 , test loss : 0.757175
epoch = 3 train_loss : 0.601612 , test loss : 0.618431
epoch = 4 train_loss : 0.526269 , test loss : 0.546513
epoch = 5 train_loss : 0.484943 , test loss : 0.508254
epoch = 6 train_loss : 0.456718 , test loss : 0.482930
epoch = 7 train_loss : 0.434904 , test loss : 0.463773
epoch = 8 train_loss : 0.417073 , test loss : 0.447909
epoch = 9 train_loss : 0.401582 , test loss : 0.433991
epoch = 10 train_loss : 0.387928 , test loss : 0.421180
epoch = 11 train_loss : 0.376064 , test loss : 0.409798
epoch = 12 train_loss : 0.365388 , test loss : 0.399940
epoch = 13 train_loss : 0.355818 , test loss : 0.390764
epoch = 14 train_loss : 0.346839 , test loss : 0.382016
epoch = 15 train_loss : 0.338422 , test loss : 0.373656
epoch = 16 train_loss : 0.330679 , test loss : 0.366201
epoch = 17 train_loss : 0.323433 , test loss : 0.359235
epoch = 18 train_loss : 0.316660 , test loss : 0.352880
e

epoch = 3 train_loss : 0.583636 , test loss : 0.643681
epoch = 4 train_loss : 0.513431 , test loss : 0.575133
epoch = 5 train_loss : 0.473913 , test loss : 0.537210
epoch = 6 train_loss : 0.449237 , test loss : 0.513542
epoch = 7 train_loss : 0.431348 , test loss : 0.496913
epoch = 8 train_loss : 0.416752 , test loss : 0.483396
epoch = 9 train_loss : 0.404074 , test loss : 0.471418
epoch = 10 train_loss : 0.392451 , test loss : 0.459390
epoch = 11 train_loss : 0.382106 , test loss : 0.449323
epoch = 12 train_loss : 0.372442 , test loss : 0.439136
epoch = 13 train_loss : 0.363195 , test loss : 0.429782
epoch = 14 train_loss : 0.354842 , test loss : 0.421697
epoch = 15 train_loss : 0.347060 , test loss : 0.414091
epoch = 16 train_loss : 0.339808 , test loss : 0.407107
epoch = 17 train_loss : 0.333109 , test loss : 0.400208
epoch = 18 train_loss : 0.326489 , test loss : 0.394170
epoch = 19 train_loss : 0.320287 , test loss : 0.387891
epoch = 20 train_loss : 0.314319 , test loss : 0.381793

epoch = 172 train_loss : 0.120981 , test loss : 0.185920
epoch = 174 train_loss : 0.120641 , test loss : 0.185668
epoch = 176 train_loss : 0.120437 , test loss : 0.185454
epoch = 181 train_loss : 0.119484 , test loss : 0.185201
epoch = 184 train_loss : 0.119030 , test loss : 0.184822
epoch = 187 train_loss : 0.118621 , test loss : 0.184799
epoch = 189 train_loss : 0.118542 , test loss : 0.184041
epoch = 192 train_loss : 0.117912 , test loss : 0.183725
epoch = 197 train_loss : 0.117262 , test loss : 0.183299
epoch = 198 train_loss : 0.117197 , test loss : 0.182915
epoch = 207 train_loss : 0.115993 , test loss : 0.182638
epoch = 208 train_loss : 0.115840 , test loss : 0.182569
epoch = 209 train_loss : 0.116011 , test loss : 0.182203
epoch = 212 train_loss : 0.115482 , test loss : 0.182177
epoch = 216 train_loss : 0.115066 , test loss : 0.182146
epoch = 218 train_loss : 0.115089 , test loss : 0.181773
epoch = 221 train_loss : 0.114529 , test loss : 0.181560
epoch = 226 train_loss : 0.1140

epoch = 110 train_loss : 0.139003 , test loss : 0.164566
epoch = 111 train_loss : 0.138357 , test loss : 0.164370
epoch = 112 train_loss : 0.137902 , test loss : 0.163800
epoch = 113 train_loss : 0.137280 , test loss : 0.163779
epoch = 114 train_loss : 0.136751 , test loss : 0.163284
epoch = 116 train_loss : 0.135627 , test loss : 0.163061
epoch = 117 train_loss : 0.135458 , test loss : 0.162951
epoch = 118 train_loss : 0.134686 , test loss : 0.162680
epoch = 119 train_loss : 0.134189 , test loss : 0.162343
epoch = 120 train_loss : 0.133763 , test loss : 0.161963
epoch = 122 train_loss : 0.132951 , test loss : 0.161896
epoch = 123 train_loss : 0.132572 , test loss : 0.161460
epoch = 124 train_loss : 0.131962 , test loss : 0.161269
epoch = 125 train_loss : 0.131581 , test loss : 0.160611
epoch = 127 train_loss : 0.130719 , test loss : 0.160542
epoch = 128 train_loss : 0.130348 , test loss : 0.160313
epoch = 129 train_loss : 0.130064 , test loss : 0.160062
epoch = 130 train_loss : 0.1296

epoch = 94 train_loss : 0.143148 , test loss : 0.169778
epoch = 95 train_loss : 0.142626 , test loss : 0.169212
epoch = 96 train_loss : 0.141995 , test loss : 0.169157
epoch = 97 train_loss : 0.141408 , test loss : 0.168478
epoch = 99 train_loss : 0.140306 , test loss : 0.167699
epoch = 100 train_loss : 0.139708 , test loss : 0.167515
epoch = 101 train_loss : 0.139331 , test loss : 0.167430
epoch = 102 train_loss : 0.139129 , test loss : 0.167363
epoch = 103 train_loss : 0.138228 , test loss : 0.166094
epoch = 104 train_loss : 0.137911 , test loss : 0.165501
epoch = 105 train_loss : 0.137314 , test loss : 0.165234
epoch = 106 train_loss : 0.136872 , test loss : 0.165146
epoch = 107 train_loss : 0.136498 , test loss : 0.164997
epoch = 108 train_loss : 0.135976 , test loss : 0.164525
epoch = 109 train_loss : 0.135578 , test loss : 0.164159
epoch = 111 train_loss : 0.134738 , test loss : 0.163316
epoch = 113 train_loss : 0.133945 , test loss : 0.163092
epoch = 114 train_loss : 0.133941 , 

epoch = 102 train_loss : 0.141434 , test loss : 0.165588
epoch = 103 train_loss : 0.140952 , test loss : 0.165348
epoch = 104 train_loss : 0.140454 , test loss : 0.165090
epoch = 105 train_loss : 0.140005 , test loss : 0.164674
epoch = 106 train_loss : 0.139464 , test loss : 0.164411
epoch = 107 train_loss : 0.139000 , test loss : 0.164074
epoch = 108 train_loss : 0.138485 , test loss : 0.163481
epoch = 111 train_loss : 0.137168 , test loss : 0.163246
epoch = 112 train_loss : 0.136872 , test loss : 0.163185
epoch = 113 train_loss : 0.136225 , test loss : 0.162948
epoch = 114 train_loss : 0.135878 , test loss : 0.162920
epoch = 115 train_loss : 0.135622 , test loss : 0.162855
epoch = 116 train_loss : 0.135013 , test loss : 0.162301
epoch = 117 train_loss : 0.134638 , test loss : 0.162295
epoch = 118 train_loss : 0.134264 , test loss : 0.162101
epoch = 119 train_loss : 0.133991 , test loss : 0.161651
epoch = 121 train_loss : 0.133007 , test loss : 0.161467
epoch = 123 train_loss : 0.1323

([0.719016969203949,
  0.5867739319801331,
  0.5062360763549805,
  0.4584769606590271,
  0.42795678973197937,
  0.40707987546920776,
  0.39174550771713257,
  0.378935843706131,
  0.3680165112018585,
  0.3581693768501282,
  0.3491467237472534,
  0.3409375250339508,
  0.33308395743370056,
  0.3259718120098114,
  0.3190900385379791,
  0.3122630715370178,
  0.306098610162735,
  0.3002399802207947,
  0.29451921582221985,
  0.28908973932266235,
  0.28387880325317383,
  0.27884432673454285,
  0.2740970849990845,
  0.26953208446502686,
  0.26525694131851196,
  0.26119303703308105,
  0.2572445571422577,
  0.253462016582489,
  0.24989835917949677,
  0.2464621663093567,
  0.2431492805480957,
  0.2399432212114334,
  0.23681803047657013,
  0.23384425044059753,
  0.23089995980262756,
  0.22798557579517365,
  0.22528132796287537,
  0.22254489362239838,
  0.21989288926124573,
  0.2172972410917282,
  0.21488076448440552,
  0.21248282492160797,
  0.2100844383239746,
  0.20776183903217316,
  0.2055449485